In [41]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [42]:
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/victorwang/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
def clean_data(folder='data',files = ['Automotive_5.json','CDs_and_Vinyl_5.json','Cell_Phones_and_Accessories_5.json','Grocery_and_Gourmet_Food_5.json','Toys_and_Games_5.json']):
    
    review_files = {}
    
    for file_name in files:
        name = file_name.split('_')[0]
        row  = pd.read_json(folder+'/'+file_name,lines=True)
        row = row.drop(columns=['verified','reviewTime','reviewerID','asin','reviewerName','unixReviewTime','vote','image','style'])
        print(f"For category {name}")
        print(row.isnull().sum())
        print(f"Dataset before dropping: {len(row)}")
        row = row.dropna()
        print(f"Dataset after dropping: {len(row)}")
        row['reviewText'] = row['reviewText']
        review_files[name] = row
    
    return review_files

In [48]:
df_collect = clean_data()

For category Automotive
overall         0
reviewText    866
summary       342
dtype: int64
Dataset before dropping: 1711519
Dataset after dropping: 1710350
For category CDs
reviewText    280
overall         0
summary       227
dtype: int64
Dataset before dropping: 1443755
Dataset after dropping: 1443288
For category Cell
overall         0
reviewText    765
summary       517
dtype: int64
Dataset before dropping: 1128437
Dataset after dropping: 1127206
For category Grocery
overall         0
reviewText    390
summary       219
dtype: int64
Dataset before dropping: 1143860
Dataset after dropping: 1143292
For category Toys
overall          0
reviewText    1175
summary        372
dtype: int64
Dataset before dropping: 1828971
Dataset after dropping: 1827456


In [49]:
df_collect['Automotive']

,overall,reviewText,summary
0,4,"After I wrote the below review, the manufactur...",Works well if you place phone in horizontally ...
1,1,It sucks barely picks up anything definitely n...,sucks
2,1,"Well to write a short one, it blew 2 fuses of ...",Defective
3,3,I have absolutely no memory of buying this but...,Looks cool! Probably works
4,5,it ok it does it job,Five Stars
...,...,...,...
1711514,5,"Fast Shipping, Works Great!",Works Great!
1711515,4,"Fit isn't great, relays are very hard to press...",Four Stars
1711516,5,I bought an auxiliary horn for my car and it n...,I bought an auxiliary horn for my car and it ...
1711517,5,Way better than stock. Nice bright and white. ...,"NICE, BRIGHT AND WHITE"


In [ ]:
""""
def read_file(filepath):
    seg_docs = {}
    
    with open(filepath, 'r') as file:
        file_dict = json.load(file)
    
    seg_docs = {}
    
    for key, text in file_dict.items():
        
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        words = word_tokenize(text)
        words_list = []
        for word in words:
            if word not in stop_words:
                words_list.append(word)
        
        seg_docs[key] = words_list
    
    return seg_docs
    """

In [39]:
"""
def strip_clean(reviews_dict):
    
    def process_entry(sentence):
        sentence = sentence.lower()
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        words_list = []
        for word in sentence:
            if word not in stop_words():
                words_list.append(word)
        return words_list
    
    stripped_dict = {}
    
    for key,df in reviews_dict.items():
        df['reviewText'] = df['reviewText'].apply(process_entry)
        df['summary'] = df['summary'].apply(process_entry)
        stripped_dict[key] = df
    
    return stripped_dict
    """

In [40]:
# df_reduced_list = strip_clean(df_collect)

TypeError: 'set' object is not callable

In [33]:
df_reduced_list['Automotive']

,overall,reviewText,summary
0,4,"after i wrote the below review, the manufactur...",works well if you place phone in horizontally ...
1,1,it sucks barely picks up anything definitely n...,sucks
2,1,"well to write a short one, it blew 2 fuses of ...",defective
3,3,i have absolutely no memory of buying this but...,looks cool! probably works
4,5,it ok it does it job,five stars
...,...,...,...
1711514,5,"fast shipping, works great!",works great!
1711515,4,"fit isn't great, relays are very hard to press...",four stars
1711516,5,i bought an auxiliary horn for my car and it n...,i bought an auxiliary horn for my car and it ...
1711517,5,way better than stock. nice bright and white. ...,"nice, bright and white"
